# Búsqueda Híbrida con Qdrant

La búsqueda vectorial basada en *embeddings* densos captura la **semántica** de los datos, lo que significa que no es necesario utilizar los mismos términos en las consultas y en los documentos para encontrar resultados relevantes. Sin embargo, históricamente también se han utilizado otros métodos que se basan en la **presencia explícita de palabras clave**. Métodos como *Bag-of-Words*, *TFIDF* y *BM25* siguen siendo útiles, y en algunos casos, se deben preferir frente a los embeddings densos.

## Vectores Dispersos (Sparse Vectors)

Sorprendentemente, la búsqueda basada en palabras clave también se implementa como una búsqueda vectorial, pero en este caso los vectores suelen ser **dispersos**. Esto significa que la mayoría de las dimensiones de estos vectores son simplemente ceros. Un valor distinto de cero en una dimensión indica la **presencia de un término** del diccionario asignado a esa posición.

En otras palabras, en los vectores dispersos, existe un diccionario en el que cada palabra o frase tiene una posición única. Dado que estos vectores tienen muchas dimensiones vacías, el diccionario puede **crecer indefinidamente**, añadiendo nuevos términos al final.

El uso de un diccionario flexible permite que los vectores dispersos sobresalgan en coincidencias exactas, ya que pueden manejar textos que para los vectores densos serían cadenas aleatorias —como **nombres propios** o **identificadores**. Los modelos de embeddings densos también utilizan un diccionario, pero una vez entrenado el modelo, **ampliarlo requiere un proceso complejo de ajuste fino (fine-tuning)**, algo poco común para el usuario promedio.

### BM25

Existen muchas formas de crear embeddings dispersos, pero **BM25** es el estándar de la industria. Su versión más conocida proviene de los años 90. Es un modelo estadístico (*no involucra redes neuronales*), lo que lo hace **rápido y ligero**. En benchmarks de búsqueda suele ser una base sólida, por lo que no debe pasarse por alto.

**BM25** significa *Best Matching 25* y fue simplemente el intento número 25 de crear una fórmula que calcule qué tan relevante es un documento con respecto a una consulta. Si te interesa la parte matemática, puedes revisar su [página en Wikipedia](https://es.wikipedia.org/wiki/Okapi_BM25).

En términos generales, BM25 es una función de ranking que ayuda a los motores de búsqueda a determinar la relevancia de un documento combinando dos conceptos clave:

1. **Frecuencia del Término (TF)**: premia a los documentos que contienen varias veces los términos de la consulta, pero con *rendimientos decrecientes* —es decir, un documento con 10 repeticiones de una palabra no es 10 veces mejor que uno con solo 1.
2. **Frecuencia Inversa de Documento (IDF)**: da mayor peso a las palabras raras y reduce la importancia de las comunes, ya que los términos raros suelen ser más informativos para distinguir resultados relevantes.

BM25 también incluye una **normalización por longitud del documento**, para evitar que los documentos más largos tengan ventaja solo por su tamaño.

En nuestro caso, utilizaremos una implementación de BM25 disponible en **FastEmbed**. Empecemos por lo básico.


## Paso 0: Configuración

Por favor, consulta el notebook [2_sematic_search.ipynb](2_sematic_search.ipynb) para configurar las librerías necesarias para interactuar con Qdrant y crear los embeddings. Del mismo modo, asegúrate de iniciar Qdrant en un contenedor Docker como se indica allí, en caso de que aún no esté en ejecución en tu máquina.

Si te saltaste nuestras lecciones anteriores, los siguientes comandos instalarán todos los paquetes necesarios y ejecutarán Qdrant en un contenedor en segundo plano.


In [ ]:
!python -m pip install -q "qdrant-client[fastembed]>=1.14.2"

In [ ]:
!docker run -d -p 6333:6333 -p 6334:6334 \
   -v "./qdrant_storage:/qdrant/storage:z" \
   qdrant/qdrant

## Paso 1: Conectarse a Qdrant

Vamos a conectarnos a Qdrant y verificar si la conexión fue exitosa listando todas las colecciones disponibles.


In [1]:
from qdrant_client import QdrantClient

client = QdrantClient("http://localhost:6333")
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='test_collection'), CollectionDescription(name='zoomcamp-rag'), CollectionDescription(name='zoomcamp-llm-hw2'), CollectionDescription(name='documentos_varios'), CollectionDescription(name='productos_alimenticios')])

## Paso 2: Búsqueda con vectores dispersos usando BM25

Vamos a utilizar el mismo conjunto de datos que antes. Descarguémoslo y carguémoslo en Qdrant, pero esta vez vamos a crear vectores dispersos **solo con BM25**.

In [2]:
import requests

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [8]:
documents_raw[0].get('documents')[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?'}

Primero necesitamos crear una colección. Qdrant se encargará de calcular los valores de IDF si lo configuramos correctamente. Esto es necesario para que BM25 funcione correctamente; de lo contrario, no se dará prioridad a las palabras raras.

In [11]:
from qdrant_client import models

# Create the collection with specified sparse vector parameters
client.create_collection(
    collection_name="zoomcamp-sparse",
    sparse_vectors_config={
        "bm25": models.SparseVectorParams(
            modifier=models.Modifier.IDF,
        )
    },
    timeout=60
)

True

FastEmbed incluye una implementación de BM25 que podemos usar como cualquier otro modelo.

In [12]:
import uuid

# Send the points to the collection
client.upsert(
    collection_name="zoomcamp-sparse",
    points=[
        models.PointStruct(
            id=uuid.uuid4().hex,
            vector={
                "bm25": models.Document(
                    text=doc["text"], 
                    model="Qdrant/bm25",
                ),
            },
            payload={
                "text": doc["text"],
                "section": doc["section"],
                "course": course["course"],
            }
        )
        for course in documents_raw
        for doc in course["documents"]
    ]
)

Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

finnish.txt: 0.00B [00:00, ?B/s]

german.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

arabic.txt: 0.00B [00:00, ?B/s]

...

spanish.txt: 0.00B [00:00, ?B/s]

norwegian.txt:   0%|          | 0.00/851 [00:00<?, ?B/s]

turkish.txt:   0%|          | 0.00/260 [00:00<?, ?B/s]

swedish.txt:   0%|          | 0.00/559 [00:00<?, ?B/s]

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

Puede que te sorprenda lo rápida que fue la operación de carga. BM25 no requiere una red neuronal, por lo que es mucho más rápido en comparación con los modelos de embeddings densos.

## Paso 3: Ejecutar búsqueda con vectores dispersos usando BM25

Ahora nuestros vectores están listos para ser consultados. Vamos a crear una función auxiliar para facilitar las búsquedas.


In [13]:
def search(query: str, limit: int = 1) -> list[models.ScoredPoint]:
    results = client.query_points(
        collection_name="zoomcamp-sparse",
        query=models.Document(
            text=query,
            model="Qdrant/bm25",
        ),
        using="bm25",
        limit=limit,
        with_payload=True,
    )

    return results.points

In [14]:
results = search("Qdrant")
results

[]

Los vectores dispersos pueden no devolver resultados si ninguna de las palabras clave de la consulta se encuentra en los documentos. No importa si existen sinónimos: la **terminología sí importa**.

In [15]:
results = search("pandas")
print(results[0].payload["text"])

You can use round() function or f-strings
round(number, 4)  - this will round number up to 4 decimal places
print(f'Average mark for the Homework is {avg:.3f}') - using F string
Also there is pandas.Series. round idf you need to round values in the whole Series
Please check the documentation
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.round.html#pandas.Series.round
Added by Olga Rudakova


Los puntajes devueltos por BM25 no se calculan usando similitud coseno, sino mediante su propia fórmula. No están acotados a un rango específico, sino que son **virtualmente ilimitados**. Veamos cómo pueden verse.

In [16]:
results[0].score

6.0392046

Esa es una observación importante antes de comenzar a implementar la búsqueda híbrida.

### Consultas en lenguaje natural

Probemos nuevamente con una pregunta aleatoria de nuestro conjunto de datos para ver qué tan bien puede funcionar la búsqueda con vectores dispersos en consultas más largas y en lenguaje natural.

In [17]:
import random
import json

random.seed(202506)

course = random.choice(documents_raw)
course_piece = random.choice(course["documents"])
print(json.dumps(course_piece, indent=2))

{
  "text": "Even though the upload works using aws cli and boto3 in Jupyter notebook.\nSolution set the AWS_PROFILE environment variable (the default profile is called default)",
  "section": "Module 4: Deployment",
  "question": "Uploading to s3 fails with An error occurred (InvalidAccessKeyId) when calling the PutObject operation: The AWS Access Key Id you provided does not exist in our records.\""
}


In [18]:
results = search(course_piece["question"])
print(results[0].payload["text"])

The trial dbt account provides access to dbt API. Job will still be needed to be added manually. Airflow will run the job using a python operator calling the API. You will need to provide api key, job id, etc. (be careful not committing it to Github).
Detailed explanation here: https://docs.getdbt.com/blog/dbt-airflow-spiritual-alignment
Source code example here: https://github.com/sungchun12/airflow-toolkit/blob/95d40ac76122de337e1b1cdc8eed35ba1c3051ed/dags/examples/dbt_cloud_example.py


### Paso 4: API Universal de Consultas de Qdrant - prefetching

El método `.query_points` de Qdrant permite construir pipelines de búsqueda en varios pasos, que pueden incorporar diferentes métodos en una sola llamada. Por ejemplo, podemos obtener algunos candidatos mediante búsqueda con vectores densos y luego reordenarlos con búsqueda dispersa, o usar un método rápido para la recuperación inicial y un método más preciso, pero lento, para el reordenamiento.

```ascii
┌────────────────┐           ┌──────────────────┐
│                │           │                  │
│  Recuperación  │ ────────► │  Reordenamiento  │
│                │           │                  │
└────────────────┘           └──────────────────┘


In [19]:
# Create the collection with both vector types
client.create_collection(
    collection_name="zoomcamp-sparse-and-dense",
    vectors_config={
        # Named dense vector for jinaai/jina-embeddings-v2-small-en
        "jina-small": models.VectorParams(
            size=512,
            distance=models.Distance.COSINE,
        ),
    },
    sparse_vectors_config={
        "bm25": models.SparseVectorParams(
            modifier=models.Modifier.IDF,
        )
    },
    timeout=60
)

True

Tenemos que subir todos los vectores a la colección recién creada.

In [20]:
client.upsert(
    collection_name="zoomcamp-sparse-and-dense",
    points=[
        models.PointStruct(
            id=uuid.uuid4().hex,
            vector={
                "jina-small": models.Document(
                    text=doc["text"],
                    model="jinaai/jina-embeddings-v2-small-en",
                ),
                "bm25": models.Document(
                    text=doc["text"], 
                    model="Qdrant/bm25",
                ),
            },
            payload={
                "text": doc["text"],
                "section": doc["section"],
                "course": course["course"],
            }
        )
        for course in documents_raw
        for doc in course["documents"]
    ]
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

onnx/model.onnx:   0%|          | 0.00/130M [00:00<?, ?B/s]

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [21]:
def multi_stage_search(query: str, limit: int = 1) -> list[models.ScoredPoint]:
    results = client.query_points(
        collection_name="zoomcamp-sparse-and-dense",
        prefetch=[
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model="jinaai/jina-embeddings-v2-small-en",
                ),
                using="jina-small",
                # Prefetch ten times more results, then
                # expected to return, so we can really rerank
                limit=(10 * limit),
            ),
        ],
        query=models.Document(
            text=query,
            model="Qdrant/bm25", 
        ),
        using="bm25",
        limit=limit,
        with_payload=True,
    )

    return results.points

In [22]:
print(json.dumps(course_piece, indent=2))

{
  "text": "Even though the upload works using aws cli and boto3 in Jupyter notebook.\nSolution set the AWS_PROFILE environment variable (the default profile is called default)",
  "section": "Module 4: Deployment",
  "question": "Uploading to s3 fails with An error occurred (InvalidAccessKeyId) when calling the PutObject operation: The AWS Access Key Id you provided does not exist in our records.\""
}


In [23]:
results = multi_stage_search(course_piece["question"])
print(results[0].payload["text"])

Problem description. How can we connect s3 bucket to MLFLOW?
Solution: Use boto3 and AWS CLI to store access keys. The access keys are what will be used by boto3 (AWS' Python API tool) to connect with the AWS servers. If there are no Access Keys how can they make sure that they have the right to access this Bucket? Maybe you're a malicious actor (Hacker for ex). The keys must be present for boto3 to talk to the AWS servers and they will provide access to the Bucket if you possess the right permissions. You can always set the Bucket as public so anyone can access it, now you don't need access keys because AWS won't care.
Read more here: https://boto3.amazonaws.com/v1/documentation/api/latest/guide/credentials.html
Added by Akshit Miglani


## Paso 5: Construyendo Búsqueda Híbrida

En sistemas productivos reales, no es necesario elegir un solo tipo de vector. Nunca se sabe qué tipo de consultas enviarán los usuarios al sistema. La búsqueda en e-commerce puede funcionar bien con búsqueda léxica sobre vectores dispersos, ya que la gente tiende a usar palabras clave, pero en sistemas conversacionales, como chatbots, las preguntas en lenguaje natural pueden ser más frecuentes. Usar un modelo para recuperar resultados y otro para reordenarlos no es la única forma de combinar vectores densos y dispersos en un mismo sistema.

La **Búsqueda Híbrida** es una técnica que combina resultados provenientes de diferentes métodos de búsqueda —por ejemplo, densos y dispersos. No existe una definición única de cómo implementarla, ya que el principal problema es mezclar resultados de métodos incompatibles. Los puntajes de búsqueda densa y dispersa no son comparables directamente, por lo que se necesita un método adicional para ordenar los resultados finales.

Hay dos términos importantes en la Búsqueda Híbrida: **fusión** y **reordenamiento (reranking)**.

### Fusión

La fusión es un conjunto de métodos que trabajan sobre los puntajes o rankings devueltos por los métodos individuales. Existen varias formas de lograr esto, pero la técnica más popular es la **Reciprocal Rank Fusion (RRF)**. Esta técnica se basa en los rankings de los documentos en cada método utilizado, y esos rankings se emplean para calcular los puntajes finales.

Normalmente no se calcula directamente estos puntajes, ya que Qdrant incluye capacidades integradas que los manejan. Sin embargo, el siguiente ejemplo puede darte una idea aproximada:

| Documento | Ranking denso | Ranking disperso | Puntaje RRF | Ranking final |
| --------- | ------------ | ---------------- | ----------- | ------------- |
| D1        | **1**        | 5                | 0.0318      | 2             |
| D2        | 2            | 4                | 0.0317      | 3             |
| D3        | 3            | 2                | 0.0320      | **1**         |
| D4        | 4            | 3                | 0.0315      | 5             |
| D5        | 5            | **1**            | 0.0318      | 2             |


In [24]:
def rrf_search(query: str, limit: int = 1) -> list[models.ScoredPoint]:
    results = client.query_points(
        collection_name="zoomcamp-sparse-and-dense",
        prefetch=[
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model="jinaai/jina-embeddings-v2-small-en",
                ),
                using="jina-small",
                limit=(5 * limit),
            ),
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model="Qdrant/bm25",
                ),
                using="bm25",
                limit=(5 * limit),
            ),
        ],
        # Fusion query enables fusion on the prefetched results
        query=models.FusionQuery(fusion=models.Fusion.RRF),
        with_payload=True
    )

    return results.points

In [25]:
results = rrf_search(course_piece["question"])
print(json.dumps(course_piece, indent=2))
print(results[0].payload["text"])

{
  "text": "Even though the upload works using aws cli and boto3 in Jupyter notebook.\nSolution set the AWS_PROFILE environment variable (the default profile is called default)",
  "section": "Module 4: Deployment",
  "question": "Uploading to s3 fails with An error occurred (InvalidAccessKeyId) when calling the PutObject operation: The AWS Access Key Id you provided does not exist in our records.\""
}
Problem description. How can we connect s3 bucket to MLFLOW?
Solution: Use boto3 and AWS CLI to store access keys. The access keys are what will be used by boto3 (AWS' Python API tool) to connect with the AWS servers. If there are no Access Keys how can they make sure that they have the right to access this Bucket? Maybe you're a malicious actor (Hacker for ex). The keys must be present for boto3 to talk to the AWS servers and they will provide access to the Bucket if you possess the right permissions. You can always set the Bucket as public so anyone can access it, now you don't need a

# Reordenamiento (Reranking)

El reordenamiento es un término más amplio relacionado con la Búsqueda Híbrida. La fusión (fusion) es una de las formas de reordenar los resultados obtenidos por múltiples métodos, pero también puedes aplicar un método más lento que no sea lo suficientemente efectivo para buscar en todos los documentos. Sin embargo, hay más aspectos involucrados. Las reglas de negocio suelen ser importantes para la recuperación, por ejemplo, se prefiere mostrar documentos que provienen de las noticias más recientes.

## Próximos pasos

Los métodos de búsqueda con vectores densos y dispersos pueden no ser suficientes en algunos casos, pero ambos son lo suficientemente rápidos para ser usados como recuperadores iniciales. Existen muchos métodos más precisos aunque más lentos, como los cross-encoders o las [representaciones multivectoriales](https://qdrant.tech/documentation/advanced-tutorials/using-multivector-representations/). Estos temas son definitivamente más avanzados y no los cubriremos ahora, pero es bueno mencionarlos para que sepas que existen.


## Ejemplo: 2

In [1]:
from qdrant_client import QdrantClient

# Instanciar el cliente para conectarse a la instancia local de Qdrant
client = QdrantClient(host="localhost", port=6333)

In [28]:
!mkdir ../datos

mkdir: cannot create directory ‘../datos’: File exists


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [29]:
!wget https://storage.googleapis.com/generall-shared-data/startups_demo.json -P ../datos

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


--2025-07-01 13:06:08--  https://storage.googleapis.com/generall-shared-data/startups_demo.json
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.128.123, 142.251.129.187, 142.251.129.155, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.128.123|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22205751 (21M) [application/json]
Saving to: ‘../datos/startups_demo.json’

startups_demo.json  100%[===================>]  21.18M   112KB/s    in 3m 54s  

2025-07-01 13:10:03 (92.5 KB/s) - ‘../datos/startups_demo.json’ saved [22205751/22205751]



In [2]:
import json

documents_for_insertion = []
metadata_payloads = []

with open("../datos/startups_demo.json", "r") as f:
    for line in f:
        obj = json.loads(line)

        # El texto principal se usará para generar los vectores
        documents_for_insertion.append(obj.get("description", ""))

        # El resto de los datos se convierte en el payload
        metadata_payloads.append(obj)

In [3]:
from qdrant_client import models

collection_name = "startups_sparse_search"
sparse_vector_name = "bm25_text"

client.create_collection(
    collection_name=collection_name,
    vectors_config={}, # No se necesitan vectores densos para este ejemplo
    sparse_vectors_config={
        sparse_vector_name: models.SparseVectorParams(
            modifier=models.Modifier.IDF
        )
    },
    timeout=60
)

True

In [18]:
import uuid

points = [
    models.PointStruct(
        id=uuid.uuid4().hex,
        vector={sparse_vector_name: models.Document(
            text=v_text, 
            model="Qdrant/bm25"
        )},
        payload=v_payload
    )
    for v_text, v_payload in zip(documents_for_insertion, metadata_payloads)
]

# Insertar en batches pequeños para evitar timeout
batch_size = 50

for i in range(0, len(points), batch_size):
    batch = points[i:i+batch_size]
    client.upsert(
        collection_name=collection_name,
        points=batch,
        wait=True  # Espera confirmación del servidor
    )

In [24]:
query_text = "A mobile application for booking travel tickets"

search_results = client.query_points(
    collection_name=collection_name,
    query=models.Document(
        text=query_text,
        model="Qdrant/bm25",
    ),
    using=sparse_vector_name,
    limit=3,
    with_payload=True,
)


for hit in search_results.points:
    print(f"Score: {hit.score}\n  Name: {hit.payload['name']}\n  Description: {hit.payload['description']}\n\n")

Score: 29.27134
  Name: Busbud
  Description: Making Bus Travel Easy.
Busbud makes bus travel easy. Busbud makes it a breeze to search, compare and book city-to-city bus tickets, anywhere in the world. Busbud.com and the free mobile application are the most comprehensive source of city-to-city bus schedules and tickets around the ...


Score: 24.56239
  Name: Distribusion Technologies
  Description: Simplifying Intercity Bus Distribution
Distribusion wants to make booking an intercity bus ticket as simple as booking a flight or hotel. We connect intercity bus operators with travel agencies, travel websites or mobility apps in a worldwide distribution network.


Score: 21.969345
  Name: Chauf4U
  Description: Online/Mobile Platform Connecting Tourism Businesses with Consumers
The company uses an online or mobile application to receive transportation requests and then sends these trip requests to their drivers and/or bases. Customers will use the website or mobile app to hail a ride, boo

## Ejemplo: 3

In [27]:
from qdrant_client import models

collection_name = "multistage_collection"
documents_for_insertion = documents_for_insertion[:1000]
metadata_payloads = metadata_payloads[:1000]

# Crear una colección con dos vectores densos con nombre
client.create_collection(
    collection_name=collection_name,
    vectors_config={
        "dense_small": models.VectorParams(size=384, distance=models.Distance.COSINE),
        "dense_large": models.VectorParams(size=768, distance=models.Distance.COSINE)
    },
    timeout=60
)

True

In [28]:
import uuid

points = [ 
    models.PointStruct(
        id=uuid.uuid4().hex,
        vector={
            "dense_small": models.Document(
                text=v_text,
                model="BAAI/bge-small-en",
            ),
            "dense_large": models.Document(
                text=v_text, 
                model="BAAI/bge-base-en",
            ),
        },
        payload=v_payload
    )
    for v_text, v_payload in zip(documents_for_insertion, metadata_payloads)
]

# Insertar en batches pequeños para evitar timeout
batch_size = 50

for i in range(0, len(points), batch_size):
    batch = points[i:i+batch_size]
    client.upsert(
        collection_name=collection_name,
        points=batch,
        wait=True  # Espera confirmación del servidor
    )

In [29]:
query_text = "What loyalty platforms are designed to strengthen the identity of local communities?"

search_results = client.query_points(
    collection_name=collection_name,
    prefetch=[
        models.Prefetch(
            query=models.Document(
                text=query_text,
                model="BAAI/bge-small-en",
            ),
            using="dense_small",
            limit=20,
        ),
    ],
    query=models.Document(
        text=query_text,
        model="BAAI/bge-base-en", 
    ),
    using="dense_large",
    limit=5,
    with_payload=True,
)

for hit in search_results.points:
    print(f"Score: {hit.score}\n  Name: {hit.payload['name']}\n  Description: {hit.payload['description']}\n\n")

Score: 0.8429852
  Name: SweetPerk
  Description: Hyper-local mobile loyalty
SweetPerk is a shopper incentive and loyalty platform that is built and branded to the unique identity of a community to promote local commerce. It partners with business improvement districts to build a locally co-branded mobile app which creates local affiliation ...


Score: 0.8353574
  Name: FanFueled
  Description: FanFueled Community Engagement Systems 
FanFueled Engagement Systems empower brands to build, congregate, mobilize and monetize fan communities better than any other loyalty platform on the market.


Score: 0.8275845
  Name: Invitation codes
  Description: The referral community
Invitation App is a social network where people post their referral codes and collect rewards on autopilot.


Score: 0.8259291
  Name: Ox&Pen
  Description: loyalty and rewads program focused on generating repeat business for local merchants
Ox&Pen is a web and mobile based universal loyalty and rewards program that pro

## Ejemplo: 4

In [21]:
import requests

# documentos de libros 
docs_url = 'https://drive.usercontent.google.com/u/0/uc?id=1zKMTAd7kXTkBuuW0JY68D4Y4wQNxMLR5&export=download'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [22]:
from qdrant_client import models

collection_name="hybrid_collection"

# Crear coleccion
client.create_collection(
    collection_name=collection_name,
    vectors_config={
        "dense_text": models.VectorParams(
            size=512,
            distance=models.Distance.COSINE,
        ),
    },
    sparse_vectors_config={
        "sparse_text": models.SparseVectorParams(
            modifier=models.Modifier.IDF,
        )
    },
    timeout=60
)

True

In [23]:
import uuid

# Cargar puntos en los vectores
client.upsert(
    collection_name=collection_name,
    points=[
        models.PointStruct(
            id=uuid.uuid4().hex,
            vector={
                "dense_text": models.Document(
                    text= f"{doc['descripcion']} {doc['clasificacion']}",
                    model="jinaai/jina-embeddings-v2-small-en",
                ),
                "sparse_text": models.Document(
                    text=f"{doc['descripcion']} {doc['clasificacion']}",
                    model="Qdrant/bm25",
                ),
            },
            payload=doc
        )
        for doc in documents
    ]
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [28]:
def busqueda(query_text, v_limit=2):
    # Busqueda hibrida
    hybrid_results = client.query_points(
        collection_name=collection_name,
        # Se definen dos prefetch, uno para cada tipo de búsqueda
        prefetch=[
            # Prefetch para la búsqueda densa
            models.Prefetch(
                query=models.Document(text=query_text, model="jinaai/jina-embeddings-v2-small-en"),
                using="dense_text", 
                limit=(v_limit * 4)),
            # Prefetch para la búsqueda dispersa
            models.Prefetch(
                query=models.Document(text=query_text, model="Qdrant/bm25"), 
                using="sparse_text", 
                limit=(v_limit * 4))
        ],
        # La consulta principal es una operación de fusión RRF    
        query=models.FusionQuery(fusion=models.Fusion.RRF),
        with_payload=True,
        limit=v_limit # Devolver los 2 mejores resultados después de la fusión
    )  
    
    return hybrid_results

In [29]:
query_text = "¿Qué novela combina elementos políticos con una visión pesimista del futuro?"

results = busqueda(query_text)

for hit in results.points:
    print(f"Score RRF: {hit.score}")
    print(f"  Titulo: {hit.payload['titulo']}, Autor: {hit.payload['autor']}, Clasficacion: {hit.payload['clasificacion']}")
    print(f"  Descripcion: {hit.payload['descripcion']}\n\n")

Score RRF: 0.5
  Titulo: La metamorfosis (97), Autor: Jane Austen, Clasficacion: Romance
  Descripcion: La historia de Elizabeth Bennet y su relación con el orgulloso Sr. Darcy en la Inglaterra del siglo XIX.


Score RRF: 0.5
  Titulo: Orgullo y prejuicio (8), Autor: George Orwell, Clasficacion: Distopía
  Descripcion: Una novela distópica sobre un régimen totalitario que vigila todos los aspectos de la vida humana.




In [30]:
query_text = "Libros que exploren la psicología del crimen y la culpa"

results = busqueda(query_text)

for hit in results.points:
    print(f"Score RRF: {hit.score}")
    print(f"  Titulo: {hit.payload['titulo']}, Autor: {hit.payload['autor']}, Clasficacion: {hit.payload['clasificacion']}")
    print(f"  Descripcion: {hit.payload['descripcion']}\n\n")

Score RRF: 0.75
  Titulo: El amor en los tiempos del cólera (43), Autor: Fiódor Dostoyevski, Clasficacion: Filosófica
  Descripcion: Un joven estudiante asesina a una anciana y enfrenta las consecuencias psicológicas y morales del crimen.


Score RRF: 0.6666667
  Titulo: El amor en los tiempos del cólera (16), Autor: Fiódor Dostoyevski, Clasficacion: Filosófica
  Descripcion: Un joven estudiante asesina a una anciana y enfrenta las consecuencias psicológicas y morales del crimen.


